# 2022 Championship EDA

In [65]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [66]:
prem_team_stats = pd.read_csv('../../footy_data/england-premier-league-teams-2021-to-2022-stats.csv')
prem_games = pd.read_csv('../../footy_data/england-premier-league-matches-2021-to-2022-stats.csv')
champ_team_stats = pd.read_csv('../../footy_data/england-championship-teams-2021-to-2022-stats.csv')
champ_games = pd.read_csv('../../footy_data/england-championship-matches-2021-to-2022-stats.csv')
league_one_team_stats = pd.read_csv('../../footy_data/england-efl-league-one-teams-2021-to-2022-stats.csv')
league_one_games = pd.read_csv('../../footy_data/england-efl-league-one-matches-2021-to-2022-stats.csv')
league_two_team_stats = pd.read_csv('../../footy_data/england-efl-league-two-teams-2021-to-2022-stats.csv')
league_two_games = pd.read_csv('../../footy_data/england-efl-league-two-matches-2021-to-2022-stats.csv')
segunda_team_stats = pd.read_csv('../../footy_data/spain-segunda-division-teams-2021-to-2022-stats.csv')
segunda_games = pd.read_csv('../../footy_data/spain-segunda-division-matches-2021-to-2022-stats.csv')

In [67]:
prem_games = prem_games.loc[prem_games['status'] == 'complete']
champ_games = champ_games.loc[champ_games['status'] == 'complete']
league_one_games = league_one_games.loc[league_one_games['status'] == 'complete']
league_two_games = league_two_games.loc[league_two_games['status'] == 'complete']
segunda_games = segunda_games.loc[segunda_games['status'] == 'complete']
#Played matches

# English Championship

In [112]:
champ_team_stats

,team_name,common_name,season,country,matches_played,matches_played_home,matches_played_away,suspended_matches,wins,wins_home,...,goals_conceded_min_81_to_90,draw_percentage_overall,draw_percentage_home,draw_percentage_away,loss_percentage_ovearll,loss_percentage_home,loss_percentage_away,over145_corners_percentage,real_xg,real_xga
0,Stoke City FC,Stoke City,2021/2022,England,15,8,7,0,6,5,...,1,27,25,29,33,13,57,27,,
1,West Bromwich Albion FC,West Bromwich Albion,2021/2022,England,15,7,8,0,8,5,...,3,27,29,25,20,0,38,7,,
2,Middlesbrough FC,Middlesbrough,2021/2022,England,15,8,7,0,6,4,...,3,20,13,29,40,38,43,13,,
3,AFC Bournemouth,AFC Bournemouth,2021/2022,England,15,7,8,0,11,5,...,0,27,29,25,0,0,0,13,,
4,Hull City AFC,Hull City,2021/2022,England,15,8,7,0,2,1,...,0,20,25,14,67,63,71,0,,
5,Swansea City AFC,Swansea City,2021/2022,Wales,15,8,7,0,5,4,...,1,33,38,29,33,13,57,7,,
6,Queens Park Rangers FC,Queens Park Rangers,2021/2022,England,15,8,7,0,6,4,...,5,27,38,14,33,13,57,13,,
7,Cardiff City FC,Cardiff City,2021/2022,Wales,15,7,8,0,3,1,...,2,20,14,25,60,71,50,7,,
8,Fulham FC,Fulham,2021/2022,England,15,8,7,0,10,6,...,2,13,13,14,20,13,29,7,,
9,Birmingham City FC,Birmingham City,2021/2022,England,15,7,8,0,5,2,...,2,27,29,25,40,43,38,7,,


In [113]:
champ_team_stats['real_xg'] = ""
champ_team_stats['real_xga'] = ""

In [114]:
total = champ_games.loc[champ_games['home_team_name'] == 'Swansea City', 'team_a_xg'].sum()
total = total + champ_games.loc[champ_games['away_team_name'] == 'Swansea City', 'team_b_xg'].sum()
total = total / (champ_team_stats.loc[champ_team_stats['common_name']=='Swansea City', 'matches_played'].iloc[0])
total
#First line -- getting their xg from home games, Second line -- getting xg from away games, third line dividing it by
# total matches played to get average xg

1.2753333333333334

In [117]:
def xg_fixer(team_name):
    total = champ_games.loc[champ_games['home_team_name'] == team_name, 'team_a_xg'].sum()
    total = total + champ_games.loc[champ_games['away_team_name'] == team_name, 'team_b_xg'].sum()
    total = total / (champ_team_stats.loc[champ_team_stats['common_name']== team_name, 'matches_played'].iloc[0])
    champ_team_stats['real_xg'] = np.where(champ_team_stats['common_name'] == team_name, total, champ_team_stats['real_xg'])

In [130]:
xg_fixer(champ_team_stats['common_name'][0])
xg_fixer(champ_team_stats['common_name'][1])
xg_fixer(champ_team_stats['common_name'][2])
xg_fixer(champ_team_stats['common_name'][3])
xg_fixer(champ_team_stats['common_name'][4])
xg_fixer(champ_team_stats['common_name'][5])
xg_fixer(champ_team_stats['common_name'][6])
xg_fixer(champ_team_stats['common_name'][7])
xg_fixer(champ_team_stats['common_name'][8])
xg_fixer(champ_team_stats['common_name'][9])
xg_fixer(champ_team_stats['common_name'][10])
xg_fixer(champ_team_stats['common_name'][11])
xg_fixer(champ_team_stats['common_name'][12])
xg_fixer(champ_team_stats['common_name'][13])
xg_fixer(champ_team_stats['common_name'][14])
xg_fixer(champ_team_stats['common_name'][15])
xg_fixer(champ_team_stats['common_name'][16])
xg_fixer(champ_team_stats['common_name'][17])
xg_fixer(champ_team_stats['common_name'][18])
xg_fixer(champ_team_stats['common_name'][19])
xg_fixer(champ_team_stats['common_name'][20])
xg_fixer(champ_team_stats['common_name'][21])
xg_fixer(champ_team_stats['common_name'][22])
xg_fixer(champ_team_stats['common_name'][23])

In [131]:
champ_team_stats

,team_name,common_name,season,country,matches_played,matches_played_home,matches_played_away,suspended_matches,wins,wins_home,...,goals_conceded_min_81_to_90,draw_percentage_overall,draw_percentage_home,draw_percentage_away,loss_percentage_ovearll,loss_percentage_home,loss_percentage_away,over145_corners_percentage,real_xg,real_xga
0,Stoke City FC,Stoke City,2021/2022,England,15,8,7,0,6,5,...,1,27,25,29,33,13,57,27,1.52133,
1,West Bromwich Albion FC,West Bromwich Albion,2021/2022,England,15,7,8,0,8,5,...,3,27,29,25,20,0,38,7,1.87133,
2,Middlesbrough FC,Middlesbrough,2021/2022,England,15,8,7,0,6,4,...,3,20,13,29,40,38,43,13,1.622,
3,AFC Bournemouth,AFC Bournemouth,2021/2022,England,15,7,8,0,11,5,...,0,27,29,25,0,0,0,13,1.46533,
4,Hull City AFC,Hull City,2021/2022,England,15,8,7,0,2,1,...,0,20,25,14,67,63,71,0,1.372,
5,Swansea City AFC,Swansea City,2021/2022,Wales,15,8,7,0,5,4,...,1,33,38,29,33,13,57,7,1.27533,
6,Queens Park Rangers FC,Queens Park Rangers,2021/2022,England,15,8,7,0,6,4,...,5,27,38,14,33,13,57,13,1.69467,
7,Cardiff City FC,Cardiff City,2021/2022,Wales,15,7,8,0,3,1,...,2,20,14,25,60,71,50,7,1.46733,
8,Fulham FC,Fulham,2021/2022,England,15,8,7,0,10,6,...,2,13,13,14,20,13,29,7,1.90267,
9,Birmingham City FC,Birmingham City,2021/2022,England,15,7,8,0,5,2,...,2,27,29,25,40,43,38,7,1.462,


In [15]:
xg_view = champ_games[['home_team_name', 'away_team_name', 'team_a_xg', 'team_b_xg', 'home_team_goal_count', 'away_team_goal_count']]
xg_view[(len(champ_games) - 50):len(champ_games)]
#We just want to see the most recent games here

,home_team_name,away_team_name,team_a_xg,team_b_xg,home_team_goal_count,away_team_goal_count
130,Peterborough United,Bristol City,1.65,1.85,2,3
131,Queens Park Rangers,Preston North End,2.02,1.37,3,2
132,West Bromwich Albion,Birmingham City,1.48,1.12,1,0
133,Fulham,Queens Park Rangers,1.74,1.08,4,1
134,Blackburn Rovers,Coventry City,1.49,1.48,2,2
135,Bristol City,AFC Bournemouth,0.79,2.46,0,2
136,Huddersfield Town,Hull City,1.54,1.24,2,0
137,Middlesbrough,Peterborough United,1.85,0.69,2,0
138,Millwall,Luton Town,1.86,0.98,0,2
139,Nottingham Forest,Blackpool,1.53,0.80,2,1


# Goal Analysis

In [16]:
season_goal_total = champ_games['total_goal_count'].sum()
season_1h_goal_total = champ_games['total_goals_at_half_time'].sum()
percent_goals_1h = ((season_1h_goal_total)/(season_goal_total)) * 100

In [17]:
print(f'There were {season_goal_total} goals scored this season')

There were 456 goals scored this season


In [18]:
print(f'There were {season_1h_goal_total} goals scored in the 1H this season, ')

There were 184 goals scored in the 1H this season, 


In [20]:
round(season_1h_goal_total/season_goal_total, 2)
#% of first half goals

0.4

In [22]:
champ_games['total_goals_at_half_time'].value_counts()

1    65
0    63
2    41
3     7
4     4
Name: total_goals_at_half_time, dtype: int64

More Columns

In [27]:
champ_games['home_xg'] = ""
champ_games['away_xg'] = ""
champ_games['home_xg_against'] = ""
champ_games['away_xg_against'] = ""
champ_games['home_total_corners'] = ""
champ_games['away_total_corners'] = ""

# Creating new columns where our data will go

Let's write a function to help fill xg in our games df

In [49]:
xg_stats = champ_team_stats[['common_name', 'xg_for_avg_overall']]
corner_stats = champ_team_stats[['common_name', 'corners_per_match']]
xga_stats = champ_team_stats[['common_name', 'xg_against_avg_overall']]

,common_name,xg_for_avg_overall
0,Stoke City,1.39
1,West Bromwich Albion,1.91
2,Middlesbrough,1.53
3,AFC Bournemouth,1.33
4,Hull City,1.24
5,Swansea City,0.91
6,Queens Park Rangers,1.61
7,Cardiff City,1.33
8,Fulham,1.95
9,Birmingham City,1.33


In [29]:
xg_stats = dict(xg_stats.values)
corner_stats = dict(corner_stats.values)
xga_stats = dict(xga_stats.values)

#By making this into dictionaries, it will be easier to call them/update them with the function

In [32]:
def fill_stats(team, games):
    games['home_xg'] = np.where(games['home_team_name'] == team, xg_stats[team], games['home_xg'])
    games['away_xg'] = np.where(games['away_team_name'] == team, xg_stats[team], games['away_xg'])
    games['home_xg_against'] = np.where(games['home_team_name'] == team, xga_stats[team], games['home_xg_against'])
    games['away_xg_against'] = np.where(games['away_team_name'] == team, xga_stats[team], games['away_xg_against'])
    games['home_total_corners'] = np.where(games['home_team_name'] == team, corner_stats[team], games['home_total_corners'])
    games['away_total_corners'] = np.where(games['away_team_name'] == team, corner_stats[team], games['away_total_corners'])
    
#Function to put team stats into games dataframe

In [46]:
fill_stats(champ_team_stats['common_name'][0], champ_games)
fill_stats(champ_team_stats['common_name'][1], champ_games)
fill_stats(champ_team_stats['common_name'][2], champ_games)
fill_stats(champ_team_stats['common_name'][4], champ_games)
fill_stats(champ_team_stats['common_name'][5], champ_games)
fill_stats(champ_team_stats['common_name'][6], champ_games)
fill_stats(champ_team_stats['common_name'][7], champ_games)
fill_stats(champ_team_stats['common_name'][8], champ_games)
fill_stats(champ_team_stats['common_name'][9], champ_games)
fill_stats(champ_team_stats['common_name'][10], champ_games)
fill_stats(champ_team_stats['common_name'][11], champ_games)
fill_stats(champ_team_stats['common_name'][12], champ_games)
fill_stats(champ_team_stats['common_name'][13], champ_games)
fill_stats(champ_team_stats['common_name'][14], champ_games)
fill_stats(champ_team_stats['common_name'][15], champ_games)
fill_stats(champ_team_stats['common_name'][16], champ_games)
fill_stats(champ_team_stats['common_name'][17], champ_games)
fill_stats(champ_team_stats['common_name'][18], champ_games)
fill_stats(champ_team_stats['common_name'][19], champ_games)
fill_stats(champ_team_stats['common_name'][20], champ_games)
fill_stats(champ_team_stats['common_name'][21], champ_games)
fill_stats(champ_team_stats['common_name'][22], champ_games)
fill_stats(champ_team_stats['common_name'][23], champ_games)

# Over/Under 2.5?

In [42]:
champ_games['over/under'] = np.where(champ_games['home_team_goal_count'] + champ_games['away_team_goal_count'] > 2.5, 1, 0)
# Adding column 0 represents under 2.5, 1 represents over 2.5

In [44]:
champ_games['over/under'].value_counts()

0    91
1    89
Name: over/under, dtype: int64

# League One

In [59]:
xg_view = league_one_games[['home_team_name', 'away_team_name', 'team_a_xg', 'team_b_xg', 'home_team_goal_count', 'away_team_goal_count']]
xg_view[(len(league_one_games) - 50):len(league_one_games)]

,home_team_name,away_team_name,team_a_xg,team_b_xg,home_team_goal_count,away_team_goal_count
133,Burton Albion,Morecambe,2.34,1.39,3,2
134,AFC Wimbledon,Sheffield Wednesday,1.25,1.78,2,2
135,Bolton Wanderers,Wigan Athletic,1.38,1.60,0,4
136,Cambridge United,Ipswich Town,1.13,1.77,2,2
137,Cheltenham Town,Accrington Stanley,1.44,1.55,1,0
138,Doncaster Rovers,Wycombe Wanderers,1.65,1.24,0,2
139,Fleetwood Town,Crewe Alexandra,1.92,1.06,3,0
140,Gillingham,Sunderland,2.72,1.61,1,2
141,Lincoln City,Charlton Athletic,2.17,0.67,2,1
142,Oxford United,Plymouth Argyle,1.41,1.56,1,3


# League Two

In [60]:
xg_view = league_two_games[['home_team_name', 'away_team_name', 'team_a_xg', 'team_b_xg', 'home_team_goal_count', 'away_team_goal_count']]
xg_view[(len(league_two_games) - 50):len(league_two_games)]

,home_team_name,away_team_name,team_a_xg,team_b_xg,home_team_goal_count,away_team_goal_count
130,Walsall,Salford City,1.92,1.09,2,1
131,Bradford City,Bristol Rovers,1.95,1.19,2,2
132,Carlisle United,Tranmere Rovers,1.82,1.49,0,1
133,Colchester United,Harrogate Town,1.13,1.67,1,0
134,Crawley Town,Sutton United,1.19,1.82,0,1
135,Exeter City,Newport County,1.73,1.60,2,2
136,Leyton Orient,Walsall,0.74,0.93,0,0
137,Northampton Town,Mansfield Town,1.14,0.97,2,0
138,Oldham Athletic,Stevenage,1.11,1.77,3,0
139,Port Vale,Barrow,1.43,1.10,3,1


# Segunda

In [61]:
xg_view = segunda_games[['home_team_name', 'away_team_name', 'team_a_xg', 'team_b_xg', 'home_team_goal_count', 'away_team_goal_count']]
xg_view[(len(segunda_games) - 50):len(segunda_games)]

,home_team_name,away_team_name,team_a_xg,team_b_xg,home_team_goal_count,away_team_goal_count
93,Real Sociedad II,SD Ponferradina,1.06,1.30,1,1
94,FC Cartagena,UD Ibiza,1.89,1.18,5,1
95,CF Fuenlabrada,Leganés,1.76,0.76,2,1
96,CD Tenerife,SD Amorebieta,1.10,1.65,2,1
97,AD Alcorcón,Burgos CF,1.55,0.50,1,0
98,Real Zaragoza,SD Huesca,1.41,1.07,0,0
99,SD Eibar,Almería,1.31,1.20,1,0
100,Girona FC,SD Huesca,2.78,1.25,1,3
101,SD Amorebieta,FC Cartagena,2.03,1.16,2,3
102,UD Las Palmas,CD Tenerife,1.48,1.18,2,1


# For Later

In [ ]:
# league[['goals_min_0_to_10', 'goals_min_11_to_20', 'goals_min_21_to_30',
#        'goals_min_31_to_40', 'goals_min_41_to_50', 'goals_min_51_to_60',
#        'goals_min_61_to_70', 'goals_min_71_to_80', 'goals_min_81_to_90',
#        'goals_min_0_to_15', 'goals_min_16_to_30', 'goals_min_31_to_45',
#        'goals_min_46_to_60', 'goals_min_61_to_75', 'goals_min_76_to_90',
#        'xg_avg_per_match']]

In [ ]:
#For new xg and xga data

#games['avg_home_xg_for_home_team'] = ""
# games['avg_away_xg_against_home_team'] = ""
# games['avg_xg_for_home_team'] = ""
# games['avg_xg_against_home_team'] = ""
# games['avg_home_xg_for_away_team'] = ""
# games['avg_away_xg_against_away_team'] = ""
# games['avg_xg_for_away_team'] = ""
# games['avg_xg_against_away_team'] = ""